In [37]:
import numpy as np
import time
import matplotlib.pyplot as plt
import treecorr

import coord
from astropy import units as u
from astropy.coordinates import SkyCoord

def get_dist_periodic(dist, boxsize):
    mask1 = dist < -boxsize/2.
    if(len(dist[mask1]) > 0):
        dist[mask1] = dist[mask1] + boxsize
    mask2 = dist > boxsize/2.
    if(len(dist[mask2]) > 0):
        dist[mask2] = dist[mask2] - boxsize
    return dist

#bin setting
nbins_pi = 20
min_pi = -100.
max_pi = 100. 
pi_bins = np.linspace(min_pi, max_pi, nbins_pi+1, endpoint=True)
delta_pi = pi_bins[1] - pi_bins[0]

nbins_rp = 20
min_rp = 1.
max_rp = 100
log_min_rp = np.log10(min_rp)
log_max_rp = np.log10(max_rp)
dlog_rp = (log_max_rp - log_min_rp) / nbins_rp
rp_bins = np.logspace(log_min_rp, log_max_rp, nbins_rp)

#read the data
boxsize = 1000. #mpc/h
zconst = boxsize*10000.

x, y, z, g12, g22 = np.loadtxt('test_data.txt', unpack=True)
z = z+zconst

#number of data
nd = len(x)

#
x1 = x
x2 = x
y1 = y 
y2 = y
z1 = z
z2 = z
w1 = np.ones(nd)
w2 = np.ones(nd)

In [38]:
#true pairs and gammaT calculation
true_npairs = np.zeros((nbins_rp, nbins_pi), dtype=int)
true_xi = np.zeros((nbins_rp, nbins_pi), dtype=complex)

t0 = time.time()
for i in range(nd):
    xdist = get_dist_periodic(x2[i]-x1, boxsize)
    ydist = get_dist_periodic(y2[i]-y1, boxsize)
    zdist = get_dist_periodic(z2[i]-z1, boxsize)
    rperp = np.sqrt(xdist**2. + ydist**2.)
    
    mask = (rperp>min_rp) & (rperp<max_rp) & (zdist>min_pi) & (zdist<max_pi)
    
    expmialpha = (xdist[mask] - 1j*ydist[mask]) / rperp[mask]
    xi = -1 * (g12[i] + 1j*g22[i]) * expmialpha**2

    rp_ind = np.floor(np.log10(rperp[mask]/ min_rp) / dlog_rp).astype(int)
    pi_ind = np.floor((zdist[mask] - min_pi)/ delta_pi).astype(int)
    
    np.add.at(true_npairs, (rp_ind, pi_ind), 1)
    np.add.at(true_xi, (rp_ind, pi_ind), xi)

xi_rppi = true_xi/1./true_npairs
t1 = time.time()
print("%f seconds passed."% (t1-t0))

0.008771 seconds passed.


/Users/jing/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in divide


In [41]:
#count pairs and gammaT with treecorr
cat1 = treecorr.Catalog(x=x1, y=y1, z=z1)
cat2 = treecorr.Catalog(x=x2, y=y2, z=z2, g1=g12, g2=g22)

#treecorr for only the first pi_bin
ng = treecorr.NGCorrelation(min_sep=min_rp, max_sep=max_rp, nbins=nbins_rp, min_rpar=pi_bins[0], max_rpar=pi_bins[1], metric='Rperp')
ng.process(cat1, cat2)

print('true_npairs = ',true_npairs[:, 0])
print('ng.pairs = ',ng.npairs)

print('true_xi = ',true_xi[:, 0])
print('ng.xi = ',ng.xi)
print('ng.xi_im = ',ng.xi_im)

('true_npairs = ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4]))
('ng.pairs = ', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  4.]))
('true_xi = ', array([ 0.00000000+0.j        ,  0.00000000+0.j        ,
        0.00000000+0.j        ,  0.00000000+0.j        ,
        0.00000000+0.j        ,  0.00000000+0.j        ,
        0.00000000+0.j        ,  0.00000000+0.j        ,
        0.00000000+0.j        ,  0.00000000+0.j        ,
        0.00000000+0.j        ,  0.00000000+0.j        ,
        0.00000000+0.j        ,  0.00000000+0.j        ,
        0.00000000+0.j        ,  0.00000000+0.j        ,
        0.00000000+0.j        ,  0.00000000+0.j        ,
        0.00000000+0.j        , -0.69700562-0.39441163j]))
('ng.xi = ', array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.

In [ ]:
'''
#testing
ra1, dec1, dcom1 = coord.CelestialCoord.xyz_to_radec(x1, y1, z1, return_r=True)
ra2, dec2, dcom2 = coord.CelestialCoord.xyz_to_radec(x2, y2, z2, return_r=True)

north_pole = coord.CelestialCoord(0*coord.radians, 90*coord.degrees)

c1 = [coord.CelestialCoord(r*coord.radians, d*coord.radians) for (r,d) in zip(ra1, dec1)]
c2 = [coord.CelestialCoord(r*coord.radians, d*coord.radians) for (r,d) in zip(ra2, dec2)]

true_npairs_spheric = np.zeros(nbins_rp, dtype=int)
true_weight_spheric = np.zeros(nbins_rp, dtype=float)
true_xi_spheric = np.zeros(nbins_rp, dtype=complex)
for i in range(nd):
    for j in range(nd):
        rsq = (x1[i]-x2[j])**2 + (y1[i]-y2[j])**2 + (z1[i]-z2[j])**2
        Lsq = ((x1[i]+x2[j])**2 + (y1[i]+y2[j])**2 + (z1[i]+z2[j])**2) / 4.
        rpar = -(dcom1[i]**2-dcom2[j]**2) / (2.*np.sqrt(Lsq))
        rsq -= rpar**2
        if(rpar<pi_bins[0]): continue
        if(rpar>pi_bins[1]): continue
        logr = 0.5 * np.log(rsq)
        k = int(np.floor((logr-log_min_rp) / dlog_rp))
        if k < 0: continue
        if k >= nbins_rp: continue

        theta2 = 90*coord.degrees - c2[j].angleBetween(c1[i], north_pole)
        expm2theta2 = np.cos(2*theta2) - 1j * np.sin(2*theta2)
        print(c1, c2)
        print(theta2)

        g2 = g12[j] + 1j * g22[j]
        g2 *= expm2theta2

        ww = w1[i] * w2[j]
        xi = -w1[i] * w2[j] * g2
                
        true_npairs_spheric[k] += 1
        true_weight_spheric[k] += ww
        true_xi_spheric[k] += xi

mask = true_weight_spheric>0.
true_xi_spheric[mask]/= true_weight_spheric[mask]
'''